# QuerySet API
- 通常在做查询操作的时候，都是通过模型名字.objects的方式进行操作。其实模型名字.objects是一个django.db.models.manager.Manager对象，而Manager这个类是一个空壳的类，他本身没有任何的属性和方法。它的方法全部都是通过Python动态添加的方式，从QuerySet类中拷贝过来的
- 我们通过实体（即数据表）映射出来的类对象和objects的结合天然拥有QuerySet类的方法，比如User.objects

# 返回新的QuerySet的方法
- 在使用QuerySet进行查找操作的时候，可以提供多种操作，比如过滤完后还要根据某个字段进行排序，那么这一系列的操作我们可以通过一个非常流畅的链式调用的方式进行。要进行链式调用，必须满足链式中的方法返回的是一个querySet对象，这样才能继续调用其他的方法，否则就不行。

### 以下是返回新的queryset对象的方法
### 1.filter：
- 将满足条件的数据提取出来，返回一个新的querySet对象，具体的filter可以提供什么条件查询，详见 django之表查询操作方法详解

### 2.exclude：
- 排除满足条件的数据，返回一个新的querySet。和filter和相反

### 3.annotate：
- 给querySet中的每一个对象都添加一个使用查询表达式（聚合函数，F表达式， Q表达式，Func表达式等）的新字段

### 4.order_by("key1", "key2")：
- 指定将查询的结果根据某个字段进行排序，如果要倒序排序的话，那么可以在这个字段的前面加一个负号。注意若使用了多个order_by的话，后面的order_by会覆盖前面的顺序。如果要实现多个关键字排序，传入多个值即可
- 可以在类Meta中定义ordering=['key1', 'key2'],能实现和order_by相同的效果
- 链式调用：result = Book.objects.annotate(order_name=Count("bookorder")).order_by("-order_name")


### 5.values：
- 用来指定在提取数据出来后，需要提取哪些字段，默认情况下会把所有的字段都提取出来，可以使用values来进行指定，并且在使用values后，提取出的querySet中的数据类型不是模型，而是在values方法中指定的字段和结果形成的字典
- values不传入参数的话默认是提取全部字段
- 示例：
        result = Magazines.objects.values("title", "content")
        for r in result:
            print(r) #r是一个字典
        result = MagazineAuthor.objects.values("name", author_name=F("authors__title"))  #链式查找

### values_list:
- values_list效果一样，只是这个方法返回的querySet中装的是元组而不是字典
- 如果只给values_list指定一个关键字字段，那么我们可以指定参数flat=True，这样返回的结果就不是一个元组，而是把只含一个元素的元祖给拆掉，得到就是这个字段的值
- 示例：
    - result = MagazineAuthor.objects.values_list("name", flat=True)

### 6.all:
- 返回所有的querySet对象

### 7.select_related（参数）:
- 在提取某个模型的数据的同时，也提前将相关的数据提取出来。这样减少数据库查询的次数
- 只能用在一对多或者一对一中或者外键的关联对象上，不能用在多对多，多对一中。比如可以提取获取文章的作者，但是不能通过作者获取这个作者的文章，或者是通过谋篇文章获取这个文章的所有标签
- 参数：
    - 当关联关系为一对一时：
        - 主表查子表，参数为 子表的小写名
        - 子表查主表，参数为 外键名
    - 当关联关系为一对多或者多对一时：
        - 主表查不了子表，无法通过 子表小写名_set的方式访问子表
        - 子表查主表：参数为 外键名
- 示例：为一对一
        1.select_related通过子表查主表，提取获取主表信息。给select_related传递的参数是外键名
        result = Magazines.objects.select_related("Mag_author")
        for r in result:
            print(r.Mag_author.name)  #在这就不用再去查询数据库了，因为select_related已经把它给提出来了
        print(connection.queries)

        2.select_related通过主表查子表，提取获取子表信息，给select_related传递的参数是 子表名小写
        result = MagazineAuthor.objects.select_related("magazines")
        for r in result:
            print(r.magazines.content)
        print(connection.queries)
- 示例：为一对多还是多对一，主表就是查不了子表，只能子表查主表
        result = Magazines.objects.select_related("company")
        for r in result:
            print(r.company.name)
        print(connection.queries)
        
        
###  8.prefetch_related（参数）:
- 这个方法和selected_related非常类似，就是在访问多个表中的数据的时候，减少查询的次数。这个方法是为了解决多对一和多对多的关系的查询问题。同时他也可以用来查询一对一和一对多的关联关系，只不过会执行两次查询，比select_related多一次
- 在查询得到结果后，不能再对结果进行条件筛选，否则prefetch_related就会失效。要给它设置筛选条件的话，得用到Prefetch()这个类
    - 导入： from django.db.models.Prefetch
            prefetch = Prefetch(参数，queryset=筛选条件)
            result = Company.objects.prefetch_related(prefetch)
    - 解释：参数即为要下面所说的参数，因为参数是对外联的那张表的连接，所以筛选条件就是对外联那张表的筛选。
        - 比如：prefetch = Prefetch("bookorder_set", queryset=BookOrder.objects.filter(price__gte=100))
        
- 参数：
    - 1.当关联关系为一对一或者一对多时，用法同select_related一模一样
    - 2.当关联关系为多对一、一对多及多对多时：
        - 主表查子表：参数为  子表名小写_set(前提是没有设置related_name参数)
        - 子表查主表：参数为  外键名
- 示例：
        # prefetch_related查询一对多
        1.主表查子表
        result = Company.objects.prefetch_related("magazines_set")
        for r in result:
            print(r.name)
            news = r.magazines_set.all()
            for new in news:
                print(new.title)
            print("===========")
        print(connection.queries)

        2.子表查主表
        result = Magazines.objects.prefetch_related("company")
        for r in result:
            print(r.title, r.company.name)
        print(connection.queries)
        
        查询多对多
        result = Tags.objects.prefetch_related("magazines_set")
        for r in result:
            print(r.tag_name)
            orders = r.magazines_set.all()
            for order in orders:
                print(order.title)
        print(connection.queries)

### defer
- 在一些表中，可能存在很多的字段，但是一些字段的数量可能是比较庞大的，但又不想提取，就可以使用defer过滤一些字段，和value类似，但是value返回的querySet中装的是字典，而defer是模型
- 注意，若过滤了某个字段，后期还想要使用的话确实可以使用，但会重新查找，影响性能

### only
- 只提取某些字段。没有提取的字段也能使用，但是也会重新查找，影响性能

### get
- 获取满足条件的数据，这个函数只能返回一条数据，并且如果给的条件有多条数据，那么这个方法就会抛出multipleobjectreturned错误。如果给的数据没有任何数据，就会返回doesnotexist错误。

### create
- 创建一条数据，并将数据保存到数据库中。
- 示例：
        xiaomi = Company.objects.create(name="小米")
        xiaomi.save()
        huawei = Company(name="华为")
        huawei.save()  #是上面的简写方式
        
### get_or_create()
- 根据某个条件进行查找，如果找到了就返回这条数据，如果没有找到，那么就创建一个
- 这个方法返回的是一个元组，元组的第一个参数是obj这个对象，第二个参数是created表示是否创建
- result = Company.objects.get_or_create(name="公司没名字")

### bulk_create(lst)
- 一次性创建多个数据，传入的参数是一个列表
- 示例：
        companies = Company.objects.bulk_create(
        [Company(name="美团"),
         Company(name="字节跳动")]
        )
        
### count()：
- 获取提取的数据的个数，如果想知道总共有多少条数据，那么建议使用count，而不是使用len()，因为count是select count(*) 来实现的，这种方式更加高效

### first()和last()
- 获取第一条和最后一条数据

### aggregate()：
- 使用聚合函数

### exists()：
- 判断某个条件的数据是否存在，如果要判断某个条件的元素是否存在，那么建议使用exists,这比使用count或者这届判断querySet更有效

### distinct:
- 去除掉那些重复的数据，这个方法如果底层数据库用的是MySQL，那么不能传递任何参数
- 需要注意的是，如果在distinct之前使用了order_by，那么因为order_by会提取order_by中指定的字段，因此再使用distinct就会根据多个字段进行唯一化，所以就不会把那些重复的数据删除掉

### update
- 执行更新操作，在SQL底层走的是update命令
- 因为走的是更新的逻辑，所以更新完成后不会执行save方法，因此不会更新auto_now设置的字段

### delete
- 删除所有满足条件的数据，删除数据的时候，要注意on_delete的处理方式

### 切片操作
- 比如：books = Book.objects.get_queryset()[1:5]， 或者Book.objects.all()[2:5]

## 注：
- 给聚合函数取的名字不能和模型中的列名重复



# django何时将queryset转换为SQL去执行
- 生成一个querySet对象并不会马上转换为SQL语句去执行
- 在以下情况下querySet会被转换为SQL语句执行
    - 1.迭代：即在遍历querySet对象的时候，首先会去执行这个SQL语句，再返回结果
    - 2.使用步长做切片操作时，querySet可以类似于列表一样做切片操作，做切片操作本身并不会执行SQL语句，但是如果在做切片操作的时候提供了步长，那么就会立马执行SQL语句。需要注意的是，做切片后不能再执行filter
    - 3.调用len函数：调用len函数用来获取querySet中总共有多少条数据时也会执行SQL语句
    - 4.调用list函数，将一个querySet对象转换为list对象时
    - 5.判断：如果对某个querySet进行判断，也会立马执行SQL语句